In [39]:
from netCDF4 import Dataset
import numpy as np
from tqdm import tqdm
import pandas as pd

def get_var(var):
    if var in ["surflw", "surfsw"]:
        return "alb"
    return var

def get_concat_axis(var):
    if var in ["surflw", "surfsw", "tstar"]:
        return 2
    return 1

date = "20110610T00"
data_shape = (70, 1500, 48)
levels_range = range(1, 49)
input_vars_m = ["tstar", "stoa", "szen", "surflw", "surfsw"]
input_vars_mn = ["p", "t", "q", "co2", "o3", "n2o", "ch4", "o2", "cfc12", "hfc134a"]
input_vars_mn1 = ["pl", "tl"]
all_input_vars = input_vars_m + input_vars_mn + input_vars_mn1
input_dataset = {}
out_config = ["ga7", "300", "260"]
out_wl = ["lw", "sw"]
output_vars = ["dflx", "uflx", "nflx", "hrts"]
output_dataset = {}

out_size = data_shape[1] * data_shape[2]
for var in tqdm(input_vars_mn + input_vars_mn1):
    arr_list = map(lambda nr: Dataset("{0}_cs_part_{1}.{2}".format(date, nr, var)).variables[var][:], levels_range)
    arr_flatten = np.concatenate(arr_list, axis=2).flatten()
    shape0 = arr_flatten.shape[0] / out_size
    for out_lev in range(shape0):
        out_key = "_".join([var, str(out_lev)])
        input_dataset[out_key] = arr_flatten[out_size * out_lev:out_size * (out_lev + 1)]
    
for var in tqdm(input_vars_m):
    arr_list = map(lambda nr: Dataset("{0}_cs_part_{1}.{2}".format(date, nr, var)).variables[get_var(var)][:], levels_range)
    input_dataset[var] = np.concatenate(arr_list, axis=get_concat_axis(var)).flatten()
    

for wl in out_wl:
    for conf in out_config:
        for var in output_vars:
            try:
                arr_list = map(
                    lambda nr: Dataset("{0}_cs_part_{1}_{2}_{3}.{4}".format(date, nr, wl, conf, var)).variables[var][:], levels_range)
                arr_flatten = np.concatenate(arr_list, axis=2).flatten()
                shape0 = arr_flatten.shape[0] / out_size
                for out_lev in range(shape0):
                    out_key = "_".join([wl, conf, var, str(out_lev)])
                    output_dataset[out_key] = arr_flatten[out_size * out_lev:out_size * (out_lev + 1)]
            except IOError:
                print "no all files for: {0}, {1}, {2}".format(wl, conf, var)
                

    


        
        
        
        

100%|██████████| 5/5 [00:00<00:00, 44.22it/s]


no all files for: lw, 260, dflx
no all files for: lw, 260, uflx
no all files for: lw, 260, nflx
no all files for: lw, 260, hrts


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in greater


no all files for: sw, 300, dflx
no all files for: sw, 300, uflx
no all files for: sw, 300, nflx
no all files for: sw, 300, hrts


In [40]:
def validate(input_dataset, output_dataset):
    ok = True
    for k, v in input_dataset.items():
        if not len(v) == out_size:
            print "ERROR, {0} field has wrong size: {1}".format(k, len(v))
            ok = False
        
    for k, v in output_dataset.items():
        if not len(v) == out_size:
            print "ERROR, {0} field has wrong size: {1}".format(k, len(v))
            ok = False
    return ok

In [43]:

def create_dataframe(input_dataset, output_dataset, write=False, out_lev="all"):
    data = {}
    seq = {}
    seq["lat"] = reduce(lambda x, y: x + y, map(lambda x: [x] * data_shape[2], range(data_shape[1])))
    seq["nr"] = range(data_shape[2]) * data_shape[1]
    input_dataset.update(seq)
    data.update(input_dataset)
    data.update(output_dataset)
    if validate(input_dataset, output_dataset):
        df = pd.DataFrame.from_dict(data)
        if write:
            df.to_csv("{0}_all_metof.csv".format(date), sep='\t', encoding='utf-8')
        return df
    return False
    
            

In [44]:
df = create_dataframe(input_dataset, output_dataset)

In [45]:
df

,cfc12_0,cfc12_1,cfc12_10,cfc12_11,cfc12_12,cfc12_13,cfc12_14,cfc12_15,cfc12_16,cfc12_17,...,tl_65,tl_66,tl_67,tl_68,tl_69,tl_7,tl_70,tl_8,tl_9,tstar
0,2.068363e-09,2.068363e-09,2.068363e-09,2.068363e-09,2.068363e-09,2.068363e-09,2.068363e-09,2.068363e-09,2.068363e-09,2.068363e-09,...,246.204926,242.758957,241.692322,241.621078,241.911591,239.672913,242.131134,223.889450,213.840881,238.750
1,9.608633e-10,9.608633e-10,9.608633e-10,9.608633e-10,9.608633e-10,9.608633e-10,9.608633e-10,9.608633e-10,9.608633e-10,9.608633e-10,...,258.793884,259.296692,259.729553,260.141907,260.513275,235.285828,260.678772,227.019913,220.805832,260.375
2,2.525779e-09,2.525779e-09,2.525779e-09,2.525779e-09,2.525779e-09,2.525779e-09,2.525779e-09,2.525779e-09,2.525779e-09,2.525779e-09,...,279.091125,279.869507,280.524292,281.059937,281.490723,236.285980,281.682495,226.748230,220.167374,285.125
3,4.025013e-09,4.025013e-09,4.025013e-09,4.025013e-09,4.025013e-09,4.025013e-09,4.025013e-09,4.025013e-09,4.025013e-09,4.025013e-09,...,284.112213,284.890533,285.541748,286.067749,286.474823,239.545212,286.649811,228.478226,217.798782,287.875
4,7.442856e-10,7.442856e-10,7.442856e-10,7.442856e-10,7.442856e-10,7.442856e-10,7.442856e-10,7.442856e-10,7.442856e-10,7.442856e-10,...,289.553833,290.336853,290.992310,291.522461,291.933044,244.772705,292.109406,238.672623,233.849121,294.750
5,5.378169e-11,5.378169e-11,5.378169e-11,5.378169e-11,5.378169e-11,5.378169e-11,5.378169e-11,5.378169e-11,5.378169e-11,5.378169e-11,...,288.296692,289.071411,289.720520,290.246094,290.655823,243.785141,290.833191,233.815186,229.031036,292.250
6,2.778194e-09,2.778194e-09,2.778194e-09,2.778194e-09,2.778194e-09,2.778194e-09,2.778194e-09,2.778194e-09,2.778194e-09,2.778194e-09,...,287.860321,288.634705,289.284454,289.811493,290.222351,247.180832,290.399933,236.495193,230.532486,293.125
7,3.168283e-09,3.168283e-09,3.168283e-09,3.168283e-09,3.168283e-09,3.168283e-09,3.168283e-09,3.168283e-09,3.168283e-09,3.168283e-09,...,278.091675,278.159302,278.470123,278.866516,279.245514,244.855881,279.424316,234.781036,230.063782,283.250
8,4.309411e-09,4.309411e-09,4.309411e-09,4.309411e-09,4.309411e-09,4.309411e-09,4.309411e-09,4.309411e-09,4.309411e-09,4.309411e-09,...,290.921906,290.946899,290.843750,290.633850,290.328125,254.688354,290.160248,247.452759,238.209854,288.375
9,4.139555e-10,4.139555e-10,4.139555e-10,4.139555e-10,4.139555e-10,4.139555e-10,4.139555e-10,4.139555e-10,4.139555e-10,4.139555e-10,...,294.652313,295.416565,296.058441,296.576447,296.972504,252.041336,297.140564,241.907455,231.236206,298.125
